In [1]:
import re
import requests
import warnings
from collections import namedtuple, defaultdict, OrderedDict
import logging

import multiprocessing as mp
import numpy as np
import pandas as pd
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display
from nltk.metrics.distance import edit_distance


pd.set_option('display.max_colwidth', 200)

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Create handlers
c_handler = logging.StreamHandler()
f_handler = logging.FileHandler('games.log', mode='a')
c_handler.setLevel(logging.WARNING)
f_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
c_format = logging.Formatter('%(message)s')
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)

## Обучим несколько моделей 

In [3]:
%%time

file_path = 'us-financial-news-articles/us-financial-news-articles.txt'

model_skipgram = fasttext.train_unsupervised(file_path, model='skipgram', dim=5)
model_cbow = fasttext.train_unsupervised(file_path, model='cbow', dim=16)
model_skipgram2 = fasttext.train_unsupervised(file_path, model='skipgram', dim=10)

CPU times: user 41min 29s, sys: 14.3 s, total: 41min 43s
Wall time: 4min 17s


In [4]:
!wc -w {file_path}

 20329509 us-financial-news-articles/us-financial-news-articles.txt


In [5]:
!head -1 {file_path}

(Updates prices, adds Trump comments) By Rodrigo Campos NEW YORK, Jan 25 (Reuters) - Mexico's peso seesawed against the dollar on Thursday as U.S. officials sent mixed signals on the greenback, while Argentina's Merval stock index broke the 35,000-point mark for the first time. Several emerging currencies hit multi-year highs against the greenback, with the dollar index languishing at more than three-year lows after U.S. Treasury Secretary Steven Mnuchin departed from traditional U.S. currency policy, saying "obviously a weaker dollar is good for us." The Mexican peso appreciated by more than 1 percent to 18.3025 earlier in the day before U.S. President Donald Trump said Mnuchin had been misinterpreted and that he ultimately wanted the dollar to be strong. Trump's comments helped the dollar to pare losses against major currencies, and the Mexican peso reversed its gains, closing down almost 0.6 percent against the greenback. Elsewhere, Colombia's peso added to Wednesday's 1.48 percent 

In [6]:
len(model_cbow.words)

135642

In [7]:
model_cbow['song']

array([ 4.881617  ,  3.679234  ,  3.7162063 , -0.229449  ,  0.89953285,
        4.7503014 , -3.3211408 ,  3.011697  ,  3.822744  ,  2.224185  ,
        0.3218044 ,  3.3189864 , -2.5510416 ,  4.9163985 ,  5.419481  ,
       -5.750706  ], dtype=float32)

In [8]:
!mkdir models
model_skipgram.save_model('models/skipgram.model')
model_skipgram2.save_model('models/skipgram2.model')
model_cbow.save_model('models/cbow.model')

mkdir: models: File exists


In [ ]:
!ls -lh models

## Реализации классов для игроков

In [10]:
class AbstractPlayer:
    def __init__(self):
        raise NotImplementedError()

    def explain(self, word, n_words):
        raise NotImplementedError()
        
    def guess(self, words, n_words):
        raise NotImplementedError()


if hasattr(model_skipgram2, 'get_nearest_neighbors'):
    print('using latest fasttext version')
    
    class LocalFasttextPlayer(AbstractPlayer):
        def __init__(self, model):
            self.model = model

        def find_words_for_sentence(self, sentence, n_closest):
            neighbours = self.model.get_nearest_neighbors(sentence)
            words = [word for similariry, word in neighbours][:n_closest]
            return words

        def explain(self, word, n_words):
            return self.find_words_for_sentence(word, n_words)

        def guess(self, words, n_words):
            return self.find_words_for_sentence(' '.join(words), n_words)
else:
    print('using older fasttext version')
    class LocalFasttextPlayer(AbstractPlayer):
        def __init__(self, model):
            self.model = model
            self.words = model.get_words()
            self.matrix = np.concatenate([model[word].reshape(1, -1) for word in self.words], axis=0)

        def find_words_for_vector(self, vector, n_closest):
            sims = cosine_similarity(vector.reshape(1, -1), self.matrix).ravel()
            word_sims = pd.Series(sims, index=self.model.get_words()).sort_values(ascending=False)
            return list(word_sims.head(n_closest).index)

        def find_words_for_sentence(self, sentence, n_closest):
            vector = self.model.get_sentence_vector(sentence)
            return self.find_words_for_vector(vector, n_closest)

        def explain(self, word, n_words):
            return self.find_words_for_sentence(word, n_words)

        def guess(self, words, n_words):
            return self.find_words_for_sentence(' '.join(words), n_words)


class RemotePlayer(AbstractPlayer):
    def __init__(self, url, timeout=1):
        self.url = url
        self.timeout = 1
        self.ping()
        
    def ping(self):
        try:
            response = requests.get(self.url)
            assert response.status_code == 200
        except Exception as exc:
            logger.warn(exc)
        
    def explain(self, word, n_words):
        try:
            response = requests.get(
                self.url + '/explain',
                {'word': word, 'n_words': n_words},
                timeout=self.timeout
            )
            word_list = response.json()
        except Exception as exc:
            logger.warning(exc)
            word_list = []
        return word_list
    
    def guess(self, words, n_words):
        try:
            response = requests.get(
                self.url + '/guess',
                {'words': words, 'n_words': n_words},
                timeout=self.timeout
            )
            word_list = response.json()
        except Exception as exc:
            logger.warning(exc)
            word_list = []
        return word_list

using latest fasttext version


In [11]:
# check locally deployed service
# remote_player = RemotePlayer('http://127.0.0.1:5000')

# check remotely deployed service
remote_player = RemotePlayer('https://obscure-everglades-02893.herokuapp.com')

print(remote_player.explain('work', 10))
print(remote_player.guess(['job', 'employee', 'office'], 5))

['work', 'discontent', 'probably:', 'lopid', 'gives', 'putty', 'refund', 'strangest', 'enuff', 'inovative']
['bars;', 'earnings', 'appellate', 'discoverd', 'phage']


In [12]:
local_player = LocalFasttextPlayer(model_skipgram)
print(local_player.explain('work', 10))
print(local_player.guess(['job', 'employee', 'office'], 5))

['pay,”', 'response."', 'favors.', 'devoted', 'ancestry', 'educated', 'choose', 'inserting', 'hierarchical', 'disjointed']
['"value"', 'timeshare', 'Healthineers,', 'Railways,', 'Postbank']


## Игра

In [82]:
class Game:
    def __init__(
        self, players, words, criteria,
        n_rounds, n_explain_words, n_guessing_words,
        random_state=None
    ):
        assert len(players) >= 2
        assert criteria in ('hard', 'soft')
        self.players = players
        self.words = words
        self.criteria = criteria
        self.n_rounds = n_rounds
        self.n_explain_words = n_explain_words
        self.n_guessing_words = n_guessing_words
        self.random_state = random_state
        
    def remove_repeated_words(self, words):
        unique_words = []
        for c in words:
            if not c in unique_words:
                unique_words.append(c)
        return unique_words
        
    def score_players(self, explainer_name, last_rounds):
        rewards = {
            player: self.n_explain_words + 1 - nround
            for player, nround in last_rounds.items()
        }
        rewards[explainer_name] = sum(rewards.values())
        return rewards

    def create_word_list(self, player, word, n_words):
        reported_words = player.explain(word, n_words)
        explain_words = reported_words[:]
        if self.criteria == 'hard':
            explain_words = explain_words[:n_words]
        # lowercase
        explain_words = [c.lower() for c in explain_words]
        # remove all symbols except letters
        explain_words = [re.sub(r'[\W\d]', '', c) for c in explain_words]
        # remove all words which have word being explained as a substring
        explain_words = [c for c in explain_words if word not in c]
        # remove all words with too small levenstein distance from the word being explained
        explain_words = [c for c in explain_words if edit_distance(word, c) > 2]
        # remove all ''
        explain_words = [c for c in explain_words if c != '']
        # remove repeated words
        explain_words = self.remove_repeated_words(explain_words)
        if self.criteria == 'soft':
            explain_words = explain_words[:n_words]
        return reported_words, explain_words
    
    def check_criteria(self, word, guessed_words):
        if self.criteria == 'soft':
            guessed = any([(word in c) and (edit_distance(word, c) < 3) for c in guessed_words])
        else:
            guessed = word in guessed_words
        return guessed

    @staticmethod
    def ask_player(player, question, word, n_words):
        method = getattr(player, question)
        return method(word, n_words)
    
    def ask_guessing_players(self, guessing_players, sentence):
        players_guesses = {}
        remote_guessing_players = []
        for player in guessing_players:
            if isinstance(player.api, RemotePlayer):
                remote_guessing_players.append(player)
            else:
                players_guesses[player.name] = player.api.guess(sentence, self.n_guessing_words)

        if len(remote_guessing_players) > 0:
            n_processes = np.fmin(len(remote_guessing_players), np.fmax(1, mp.cpu_count() - 1))
            pool = mp.Pool(n_processes)
            remote_players_guesses = pool.starmap(
                self.ask_player,
                [(p.api, 'guess', sentence, self.n_guessing_words) for p in remote_guessing_players]
            )
            pool.close()

            players_guesses.update(
                zip([player.name for player in remote_guessing_players], remote_players_guesses)
            )
        return players_guesses

    def play_round(self, explaining_player, guessing_players, word, sentence):
        game_round = OrderedDict()
        results = {}
        logger.info(f"HOST: {sentence}")
        game_round.update({f'Explanation for "{word}" ({explaining_player.name})': sentence})

        if False:
            for player in guessing_players:
                guessed_words = player.api.guess(sentence, self.n_guessing_words)
                guessed = self.check_criteria(word, guessed_words)
                results[player.name] = guessed
                logger.info(f'GUESSING PLAYER ({player.name}) to HOST: {guessed_words}')
                logger.info(f'HOST: {guessed}')
                game_round.update({f'Guess ({player.name})': guessed_words})
        else:
            players_guesses = self.ask_guessing_players(guessing_players, sentence)

            for player in guessing_players:
                guessed_words = players_guesses[player.name]
                guessed = self.check_criteria(word, guessed_words)
                results[player.name] = guessed
                logger.info(f'GUESSING PLAYER ({player.name}) to HOST: {guessed_words}')
                logger.info(f'HOST: {guessed}')
                game_round.update({f'Guess ({player.name})': guessed_words})

        return game_round, results

    def play(self, explaining_player, guessing_players, word, criteria):

        logger.info(f'HOST to EXPLAINING PLAYER ({explaining_player.name}): the word is "{word}"')

        reported_words, guessing_by = self.create_word_list(explaining_player.api, word, self.n_explain_words)
        logger.info(f'EXPLAINING PLAYER ({explaining_player.name}) to HOST: my wordlist is {reported_words}')
        logger.info(f'HOST TO EXPLAINING PLAYER ({explaining_player.name}): cleaning your word list. Now the list is {guessing_by}')

        df = []
        success_rounds = {}
        for iround in range(1, len(guessing_by) + 1):
            if len(guessing_players) == 0:
                break
            logger.info(f'\n===ROUND {iround}===\n')
            game_round, results_round = self.play_round(
                explaining_player=explaining_player,
                guessing_players=guessing_players,
                word=word,
                sentence=guessing_by[:iround],
            )
            for player in guessing_players[:]:
                if (player.name not in success_rounds) and results_round.get(player.name, False):
                    success_rounds[player.name] = iround
                    guessing_players = [p for p in guessing_players if p != player]
            df.append(game_round)
        
        df = pd.DataFrame(df)
        scores = self.score_players(explaining_player.name, success_rounds)
        
        return df, scores
    
    def get_words(self, complete):
        if not complete:
            words = self.words[:]
            np.random.seed(self.random_state)
            np.random.shuffle(words)
        else:
            words = []
            for word in self.words:
                words.extend([word] * len(self.players))
        return words
    
    def get_n_rounds(self, complete):
        if complete:
            return len(self.words)
        else:
            return self.n_rounds
        
    @staticmethod
    def set_console_logging_level(verbose):
        if verbose == 'print_logs':
            console_logging_level = logging.INFO
        else:
            console_logging_level = logging.WARNING
        c_handler.setLevel(console_logging_level)

    def run(self, verbose=False, complete=False):
        self.set_console_logging_level(verbose)
        
        self.run_words = self.get_words(complete=complete)
        self.run_rounds = self.get_n_rounds(complete=complete)
            
        igame = 0
        scores = []
        scores_status = defaultdict(int)
        for r in range(self.run_rounds):
            for explaining_player in self.players:
                guessing_players = [p for p in self.players if p != explaining_player]
                try:
                    word = self.run_words[igame]
                except IndexError:
                    break
                df, score = self.play(
                    explaining_player, guessing_players, word,
                    criteria=self.criteria
                )
                scores.append(score)
                scores_status[(explaining_player.name, 'explaining')] += score.get(explaining_player.name, 0)
                for player in guessing_players:
                    scores_status[(player.name, 'guessing')] += score.get(player.name, 0)
                igame += 1
                logger.info(f'\n\nSCORES: {score}')
                if verbose:
                    display(df)

        self.scores = pd.DataFrame(scores).fillna(0)
        self.scores.index.name = 'game'
        
        self.scores_status = pd.Series(scores_status).unstack()        

    def report_results(self, each_game=False):
        if each_game:
            print('=== Team scores in each game ===')
            display(self.scores)
        logger.debug(self.scores)
        
        print('=== Team scores, summary ===')
        self.summary = self.scores_status
        self.summary['total'] = self.scores.sum(axis=0)
        self.summary.sort_values('total', ascending=False)
        display(self.summary)
        logger.debug(self.summary)


player = namedtuple('Player', ['name', 'api'])

N_EXPLAIN_WORDS = 10
N_GUESSING_WORDS = 5
N_ROUNDS = 1
CRITERIA = 'soft'

PLAYERS = [
    player('HerokuOrg team', RemotePlayer('https://obscure-everglades-02893.herokuapp.com')),
    player('GKEOrg team', RemotePlayer('http://104.154.210.36')),
    player('skipgram team', LocalFasttextPlayer(model_skipgram)),
    player('skipgram2 team', LocalFasttextPlayer(model_skipgram2)),
    player('cbow team', LocalFasttextPlayer(model_cbow))
]

WORDS = ['dollar', 'percent', 'billion', 'money']

game = Game(PLAYERS, WORDS, CRITERIA, N_ROUNDS, N_EXPLAIN_WORDS, N_GUESSING_WORDS, random_state=0)
game.run(verbose='print_logs', complete=False)

HOST to EXPLAINING PLAYER (HerokuOrg team): the word is "billion"
EXPLAINING PLAYER (HerokuOrg team) to HOST: my wordlist is ['billion', 'dings', '>near', 'pellets', 'prelude', '100th', 'ammo', 'calibre', 'mile', 'corollas']
HOST TO EXPLAINING PLAYER (HerokuOrg team): cleaning your word list. Now the list is ['dings', 'near', 'pellets', 'prelude', 'th', 'ammo', 'calibre', 'mile', 'corollas']

===ROUND 1===

HOST: ['dings']
GUESSING PLAYER (GKEOrg team) to HOST: ['endings', 'landings', 'green"', 'readings', 'Stocks-Factors']
HOST: False
GUESSING PLAYER (skipgram team) to HOST: ['months,', 'thirds', 'while', "year's", 'short-listed']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ['passengers’', 'flowed', 'tremors', 'tanker.', 'chilling']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['readings', 'slides,', 'spreading.', 'records', 'darlings']
HOST: False

===ROUND 2===

HOST: ['dings', 'near']
GUESSING PLAYER (GKEOrg team) to HOST: ['near', 'river', 'deltas', 'Saqba', 'area']


,"Explanation for ""billion"" (HerokuOrg team)",Guess (GKEOrg team),Guess (skipgram team),Guess (skipgram2 team),Guess (cbow team)
0,[dings],"[endings, landings, green"", readings, Stocks-F...","[months,, thirds, while, year's, short-listed]","[passengers’, flowed, tremors, tanker., chilling]","[readings, slides,, spreading., records, darli..."
1,"[dings, near]","[near, river, deltas, Saqba, area]","['17,, ETFs, Accuweather., occupant, Us']","[couples., ¢, mega-projects, caffeinated, popu...","[downtown., storming, “near, sub-freezing, ""near]"
2,"[dings, near, pellets]","[near, pellets, deltas, 900-square-nautical-mi...","[Disney/Fox, playability, venues,, leaders', p...","[sprouted, mega-projects, frozen., rhinos, und...","[pellets, parasite, trappings, furnishings, da..."
3,"[dings, near, pellets, prelude]","[prelude, Strait., Kholmsk,, spines, militarized]","[townhouses, colossus, brews, Frappuccinos, do...","[couples., symbolized, home-country, tolling, ...","[casual, mortuary, distort, fading,, draws,]"
4,"[dings, near, pellets, prelude, th]","[Strait., Kholmsk,, near, desolate, Plunge]","[colossus, townhouses, brews, dollars,"", roll-up]","[¢, tolling, couples., Alorica, symbolized]","[mortuary, recording,, torrent, casual, draws,]"
5,"[dings, near, pellets, prelude, th, ammo]","[900-square-nautical-mile, Saada,, Bundaberg, ...","[colossus, townhouses, (by, Samsung,, goals:]","[¢, couples., Alorica, tolling, Tabbaa]","[fading,, casual, distort, circulating,, draws,]"
6,"[dings, near, pellets, prelude, th, ammo, cali...","[SAS:, 900-square-nautical-mile, CA10YT=RR, Pa...","[port-trucking, roll-up, Piecyk, Larsen’s, Ren...","[Alorica, Tabbaa, portends, hindrance, tolling]","[distort, intersex, paintings, distorts, distr..."
7,"[dings, near, pellets, prelude, th, ammo, cali...","[900-square-nautical-mile, CA10YT=RR, SAS:, Kh...","[port-trucking, Larsen’s, situation:, roll-up,...","[Alorica, portends, symbolized, Tabbaa, convol...","[distort, manifestation, intersex, distorts, c..."
8,"[dings, near, pellets, prelude, th, ammo, cali...","[900-square-nautical-mile, CA10YT=RR, Kholmsk,...","[Larsen’s, roll-up, port-trucking, WalletHub's...","[non-starter, Alorica, portends, home-country,...","[distort, distorts, manifestation, outreach,, ..."


HOST to EXPLAINING PLAYER (GKEOrg team): the word is "money"
EXPLAINING PLAYER (GKEOrg team) to HOST: my wordlist is ['money', 'money,', 'funds', 'laundering', 'money?', 'money.', 'launder', 'dollars', 'funds)', 'laundering.']
HOST TO EXPLAINING PLAYER (GKEOrg team): cleaning your word list. Now the list is ['funds', 'laundering', 'launder', 'dollars']

===ROUND 1===

HOST: ['funds']
GUESSING PLAYER (HerokuOrg team) to HOST: ['funds', 'quasars', 'near-earth', 'steroids', 'underground']
HOST: False
GUESSING PLAYER (skipgram team) to HOST: ['750,000', 'repaid', 'storm-impacted', 'owed,', 'comparison,']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ['debt-to-equity', 'debt', 'funds,', 'syndicates', 'debt-for-equity']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['funds,', 'funds’', "funds'", 'lenders', 'funds.']
HOST: False

===ROUND 2===

HOST: ['funds', 'laundering']
GUESSING PLAYER (HerokuOrg team) to HOST: ['shields', 'lobbied', 'labors', 'inspector', '>center']
HOST: Fals

,"Explanation for ""money"" (GKEOrg team)",Guess (HerokuOrg team),Guess (skipgram team),Guess (skipgram2 team),Guess (cbow team)
0,[funds],"[funds, quasars, near-earth, steroids, undergr...","[750,000, repaid, storm-impacted, owed,, compa...","[debt-to-equity, debt, funds,, syndicates, deb...","[funds,, funds’, funds', lenders, funds.]"
1,"[funds, laundering]","[shields, lobbied, labors, inspector, >center]","[forth,, table"", “illicit, constituencies, made]","[money-laundering, laundering., creditors’, so...","[money-laundering, laundering, anti-money-laun..."
2,"[funds, laundering, launder]","[midsummer, lojack, vogue, supercenter, >[dd]]","[“local, CFIUS, mandated,, consultation, Hongs...","[laundering., launder, laundering,, anti-money...","[launder, laundering., money-laundering, laund..."
3,"[funds, laundering, launder, dollars]","[victories, neurosurgeon, spoon, vogue, hollowed]","[landline, Fujitsu’s, Gold’s, Son's, carriers.]","[policies:, issues:, Others,, anti-money, prio...","[money-laundering, laundering., funds’, launde..."


HOST to EXPLAINING PLAYER (skipgram team): the word is "percent"
EXPLAINING PLAYER (skipgram team) to HOST: my wordlist is ['-API', 'billion),', '$570', '($37', '190,000', '$1.2045.', 'ACWI', '$68.27', '($2.75', 'month-on-month']
HOST TO EXPLAINING PLAYER (skipgram team): cleaning your word list. Now the list is ['api', 'billion', 'acwi', 'monthonmonth']

===ROUND 1===

HOST: ['api']
GUESSING PLAYER (HerokuOrg team) to HOST: ['api', 'jpeg-based', 'nnnn', 'postscript-based', 'xoff']
HOST: False
GUESSING PLAYER (GKEOrg team) to HOST: ['Knicks,', 'Rondae', 'DeMarre', 'Caris', 'child,']
HOST: False
GUESSING PLAYER (skipgram2 team) to HOST: ['four,', 'free-throw', 'dunks', 'wickets.', 'Ashes,']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['go.', 'apiece', 'hook', 'throw.', 'apiece,']
HOST: False

===ROUND 2===

HOST: ['api', 'billion']
GUESSING PLAYER (HerokuOrg team) to HOST: ['microtimes', 'pending', 'intersection', 'partition', 'intersect']
HOST: False
GUESSING PLAYER (GKEOrg team) 

,"Explanation for ""percent"" (skipgram team)",Guess (HerokuOrg team),Guess (GKEOrg team),Guess (skipgram2 team),Guess (cbow team)
0,[api],"[api, jpeg-based, nnnn, postscript-based, xoff]","[Knicks,, Rondae, DeMarre, Caris, child,]","[four,, free-throw, dunks, wickets., Ashes,]","[go., apiece, hook, throw., apiece,]"
1,"[api, billion]","[microtimes, pending, intersection, partition,...","[billion, million, ($600, ($1.83, ($1.17]","[135,000, ($8.3, 2,700, BMN, 88]","[billion, $18-billion, $13.6-billion, billion-..."
2,"[api, billion, acwi]","[ecg, heightfield, emi, terminator, 95-97%]","[MTV., billion, ($600, teenaged, trinity]","[($4.6, 45, ($48, BMN, ($940]","[$18-billion, billion:, billion-plus, billion,..."
3,"[api, billion, acwi, monthonmonth]","[calender, bmj, emi, sundance, pasp]","[MTV., $17.42., ($88.39, 402.95, 17.92]","[trln, 2,700-mark, $65.56, $69.02, Sensex]","[billionth, $1.2, billion-a-year, $17.2, $1.4]"


HOST to EXPLAINING PLAYER (skipgram2 team): the word is "dollar"
EXPLAINING PLAYER (skipgram2 team) to HOST: my wordlist is ['greenback,', 'dollar,', '.DXY,', '.DXY.', 'greenback', 'greenback.', 'dollar.', 'Treasuries.', 'euro', 'firmer']
HOST TO EXPLAINING PLAYER (skipgram2 team): cleaning your word list. Now the list is ['greenback', 'dxy', 'treasuries', 'euro', 'firmer']

===ROUND 1===

HOST: ['greenback']
GUESSING PLAYER (HerokuOrg team) to HOST: ['parabellum', 'bort', 'soar', 'hooper', 'korg']
HOST: False
GUESSING PLAYER (GKEOrg team) to HOST: ['greenback', 'Greenback', 'dollar', '.DXY', 'dollar,']
HOST: True
GUESSING PLAYER (skipgram team) to HOST: ['trims', 'euro', 'weighed', 'mid-2015', 'Treasuries.']
HOST: False
GUESSING PLAYER (cbow team) to HOST: ['greenback,', 'greenback.', "greenback's", 'greenback’s', 'dollar']
HOST: True

===ROUND 2===

HOST: ['greenback', 'dxy']
GUESSING PLAYER (HerokuOrg team) to HOST: ['cm^3', 'mqcnaitfksqaaaeeakceejwi9f5kmjykp0logc5dghrpbmy2xhoo8kpeh

,"Explanation for ""dollar"" (skipgram2 team)",Guess (HerokuOrg team),Guess (GKEOrg team),Guess (skipgram team),Guess (cbow team)
0,[greenback],"[parabellum, bort, soar, hooper, korg]","[greenback, Greenback, dollar, .DXY, dollar,]","[trims, euro, weighed, mid-2015, Treasuries.]","[greenback,, greenback., greenback's, greenbac..."
1,"[greenback, dxy]","[cm^3, mqcnaitfksqaaaeeakceejwi9f5kmjykp0logc5...",NaN,"[shorter-dated, bond, Treasuries., rose,, drop]",NaN
2,"[greenback, dxy, treasuries]","[timeshare, hesperidin, showme, -jules, upon:]",NaN,"[bond, lags, shorter-dated, [.N], decade-low]",NaN
3,"[greenback, dxy, treasuries, euro]","[cadillac, zorro, jps, vines, reaction_diffusion]",NaN,"[rial, (JGBs), five-year,, steady, boosted]",NaN
4,"[greenback, dxy, treasuries, euro, firmer]","[1-26-93, vines, zildjian, ec, >cam]",NaN,"[IEA, sectoral, [.N], benchmarking, halve]",NaN


In [83]:
game.report_results(each_game=True)

=== Team scores in each game ===


,GKEOrg team,HerokuOrg team,cbow team,skipgram team,skipgram2 team
game,,,,,
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,10.0,0.0,10.0,0.0,20.0


=== Team scores, summary ===


,explaining,guessing,total
GKEOrg team,0.0,10.0,10.0
HerokuOrg team,0.0,0.0,0.0
cbow team,NaN,10.0,10.0
skipgram team,0.0,0.0,0.0
skipgram2 team,20.0,0.0,20.0


In [88]:
PLAYERS = [
    player('skipgram team', LocalFasttextPlayer(model_skipgram)),
    player('skipgram2 team', LocalFasttextPlayer(model_skipgram2)),
    player('cbow team', LocalFasttextPlayer(model_cbow))
]

N_WORDS = 50

for vocabulary_path in [
    'vocabulary/topic_finance_frequent_words.txt',
    'vocabulary/verbs_top_50.txt',
    'vocabulary/nouns_top_50.txt',
    'vocabulary/adjectives_top_50.txt'
]:
    print(vocabulary_path)
    with open(vocabulary_path) as f:
        WORDS = f.readlines()
        WORDS = [word.strip() for word in WORDS][:N_WORDS]

    game_start = pd.Timestamp.now()
    game = Game(PLAYERS, WORDS, CRITERIA, N_WORDS // len(PLAYERS), N_EXPLAIN_WORDS, N_GUESSING_WORDS)
    game.run(verbose=False, complete=False)
    game.report_results(each_game=False)
    game_end = pd.Timestamp.now()
    display(game_end - game_start)

vocabulary/topic_finance_frequent_words.txt
=== Team scores, summary ===


,explaining,guessing,total
cbow team,36,14,50.0
skipgram team,9,9,18.0
skipgram2 team,14,36,50.0


Timedelta('0 days 00:00:03.013426')

vocabulary/verbs_top_50.txt
=== Team scores, summary ===


,explaining,guessing,total
cbow team,30,34,64.0
skipgram team,20,10,30.0
skipgram2 team,44,50,94.0


...

,explaining,guessing,total
cbow team,18,30,48.0
skipgram team,0,14,14.0
skipgram2 team,36,10,46.0


Timedelta('0 days 00:00:03.481774')

vocabulary/adjectives_top_50.txt
=== Team scores, summary ===


,explaining,guessing,total
cbow team,20,0,20.0
skipgram team,0,10,10.0
skipgram2 team,10,20,30.0


Timedelta('0 days 00:00:03.347001')